In [24]:
import pandas as pd

# text processing
import re

In [10]:
df = pd.read_csv('../Data/full_model_spec_sales_df_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       4092 non-null   int64  
 1   Total_Sales      4092 non-null   float64
 2   Year             4092 non-null   int64  
 3   cleaned_name     4092 non-null   object 
 4   curb_weight_lbs  2116 non-null   float64
 5   speed_sec        3142 non-null   float64
 6   horsepower_hp    3408 non-null   float64
 7   width_in         3052 non-null   float64
 8   msrp             3526 non-null   float64
 9   pass_capacity    3253 non-null   float64
 10  doors            3253 non-null   float64
 11  wheelbase_in     3539 non-null   float64
 12  height_in        3320 non-null   float64
 13  tank_cap_gal     3525 non-null   float64
 14  length_in        3017 non-null   float64
 15  vol_cubft        2428 non-null   float64
 16  mpg_comb         3539 non-null   float64
 17  drivetrain    

### load additonal datset

In [75]:
specs_df = pd.read_csv('/Users/racheldilley/Documents/car-sales-predictor-data/cars.csv')
specs_df['Year'] = specs_df['Identification.Year']
specs_df.drop(['Engine Information.Engine Type', 'Fuel Information.City mpg', 'Fuel Information.Highway mpg',
              'Identification.ID', 'Identification.Make', 'Identification.Year', 'Engine Information.Number of Forward Gears'], axis=1, inplace=True)
specs_df.drop_duplicates(['Identification.Model Year'], keep='first', inplace=True)
specs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918 entries, 0 to 5074
Data columns (total 12 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Dimensions.Height                                918 non-null    int64 
 1   Dimensions.Length                                918 non-null    int64 
 2   Dimensions.Width                                 918 non-null    int64 
 3   Engine Information.Driveline                     918 non-null    object
 4   Engine Information.Hybrid                        918 non-null    bool  
 5   Engine Information.Transmission                  918 non-null    object
 6   Fuel Information.Fuel Type                       918 non-null    object
 7   Identification.Classification                    918 non-null    object
 8   Identification.Model Year                        918 non-null    object
 9   Engine Information.Engine Statistics.Horse

In [76]:
def clean_name(name):
    name = name[4:]
    name = name.lower()
    name = re.sub(r'[^\w\s]', ' ', name)
   
    return name.strip()

In [77]:
specs_df['cleaned_name'] = specs_df.apply(lambda x: clean_name(x['Identification.Model Year']), axis=1)


df2 = df.merge(specs_df, how = 'left',on=["cleaned_name", 'Year'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4093 entries, 0 to 4092
Data columns (total 32 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       4093 non-null   int64  
 1   Total_Sales                                      4093 non-null   float64
 2   Year                                             4093 non-null   int64  
 3   cleaned_name                                     4093 non-null   object 
 4   curb_weight_lbs                                  2116 non-null   float64
 5   speed_sec                                        3143 non-null   float64
 6   horsepower_hp                                    3409 non-null   float64
 7   width_in                                         3053 non-null   float64
 8   msrp                                             3527 non-null   float64
 9   pass_capacity                 

In [78]:
import math
def fill_null_values(x, y):
    try:
        if math.isnan(x):
            return y
        else:
            return x
    except:
        return x

drop_columns = ['Identification.Model Year']
df3 = df2.copy()
df3.drop(drop_columns, axis=1, inplace=True )

column_pairs = {'heightIN': ['height_in','Dimensions.Height'], 'lengthIN': ['length_in', 'Dimensions.Length'], 
               'widthIN': ['width_in', 'Dimensions.Width'], 'drive': ['drivetrain', 'Engine Information.Driveline'],
               'trans': ['transmission', 'Engine Information.Transmission'], 'fuel_type': ['fuel', 'Fuel Information.Fuel Type'],
               'epa_class':['class', 'Identification.Classification'], 'horsepowerHP': ['horsepower_hp', 'Engine Information.Engine Statistics.Horsepower'] }

for key, value in column_pairs.items():
    df3[key] = df3.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    df3.drop(value, axis=1, inplace=True)

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4093 entries, 0 to 4092
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   4093 non-null   int64  
 1   Total_Sales                                  4093 non-null   float64
 2   Year                                         4093 non-null   int64  
 3   cleaned_name                                 4093 non-null   object 
 4   curb_weight_lbs                              2116 non-null   float64
 5   speed_sec                                    3143 non-null   float64
 6   msrp                                         3527 non-null   float64
 7   pass_capacity                                3254 non-null   float64
 8   doors                                        3254 non-null   float64
 9   wheelbase_in                                 3540 non-null   float64
 10  

In [79]:
df4 = df3.merge(specs_df, how = 'left',on=["cleaned_name"])
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7209 entries, 0 to 7208
Data columns (total 35 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       7209 non-null   int64  
 1   Total_Sales                                      7209 non-null   float64
 2   Year_x                                           7209 non-null   int64  
 3   cleaned_name                                     7209 non-null   object 
 4   curb_weight_lbs                                  4027 non-null   float64
 5   speed_sec                                        5778 non-null   float64
 6   msrp                                             6429 non-null   float64
 7   pass_capacity                                    6021 non-null   float64
 8   doors                                            6021 non-null   float64
 9   wheelbase_in                  

In [80]:
drop_columns = ['Identification.Model Year', 'Year_y', 'Year_x', 'Unnamed: 0']
df5 = df4.copy()
df5['Year'] = df5['Year_x']
df5.drop(drop_columns, axis=1, inplace=True )

column_pairs = {'height_in': ['heightIN','Dimensions.Height'], 'length_in': ['lengthIN', 'Dimensions.Length'], 
               'width_in': ['widthIN', 'Dimensions.Width'], 'drivetrain': ['drive', 'Engine Information.Driveline'],
               'transmission': ['trans', 'Engine Information.Transmission'], 'fuel': ['fuel_type', 'Fuel Information.Fuel Type'],
               'class':['epa_class', 'Identification.Classification'], 'horsepower_hp': ['horsepowerHP', 'Engine Information.Engine Statistics.Horsepower'],
                'hybrid': ['Engine Information.Hybrid_x','Engine Information.Hybrid_y'], 'torque':
                ['Engine Information.Engine Statistics.Torque_x', 'Engine Information.Engine Statistics.Torque_y'] }

for key, value in column_pairs.items():
    df5[key] = df5.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    df5.drop(value, axis=1, inplace=True)

df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7209 entries, 0 to 7208
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Sales      7209 non-null   float64
 1   cleaned_name     7209 non-null   object 
 2   curb_weight_lbs  4027 non-null   float64
 3   speed_sec        5778 non-null   float64
 4   msrp             6429 non-null   float64
 5   pass_capacity    6021 non-null   float64
 6   doors            6021 non-null   float64
 7   wheelbase_in     6455 non-null   float64
 8   tank_cap_gal     6449 non-null   float64
 9   vol_cubft        4494 non-null   float64
 10  mpg_comb         6429 non-null   float64
 11  Year             7209 non-null   int64  
 12  height_in        6646 non-null   float64
 13  length_in        6552 non-null   float64
 14  width_in         6625 non-null   float64
 15  drivetrain       6819 non-null   object 
 16  transmission     6522 non-null   object 
 17  fuel          

In [81]:
df5.drop_duplicates(['cleaned_name', 'Year'], keep='first', inplace=True)
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4092 entries, 0 to 7206
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Sales      4092 non-null   float64
 1   cleaned_name     4092 non-null   object 
 2   curb_weight_lbs  2116 non-null   float64
 3   speed_sec        3142 non-null   float64
 4   msrp             3526 non-null   float64
 5   pass_capacity    3253 non-null   float64
 6   doors            3253 non-null   float64
 7   wheelbase_in     3539 non-null   float64
 8   tank_cap_gal     3525 non-null   float64
 9   vol_cubft        2428 non-null   float64
 10  mpg_comb         3539 non-null   float64
 11  Year             4092 non-null   int64  
 12  height_in        3529 non-null   float64
 13  length_in        3435 non-null   float64
 14  width_in         3508 non-null   float64
 15  drivetrain       3702 non-null   object 
 16  transmission     3405 non-null   object 
 17  fuel          

In [90]:
df5['fuel_type'] = df5.apply(lambda x: 'Hybrid' if x['hybrid'] == True else x['fuel'], axis=1)
df5.drop(['fuel','torque'], axis=1, inplace=True)

In [91]:
#save as csv to manually add in missing data
df5.to_csv('../Data/full_model_spec_sales_df_cleaned2.csv')

In [86]:
null_cols = ['height_in', 'length_in', 'width_in', 'drivetrain', 'transmission', 'fuel', 'class', 'horsepower_hp',
                'hybrid', 'torque']
null_df = df5[df5[null_cols].isna().all(1)] #.groupby('cleaned_name')
null_df['cleaned_name'].value_counts()[:15]
# null_weight = df5[df5[null_cols].isnull()]
# null_weight

chrysler town country     10
mercedes benz slk          7
volkswagen golf rabbit     6
suzuki aerio sx4           6
chevrolet caprice ppv      6
mercury mountaineer        6
infiniti g                 6
mercedes benz clk          6
lexus sc430                6
lexus ct200h               5
chevrolet bolt             5
suzuki forenza reno        5
ford c max                 5
audi a4 allroad            5
mazda b                    5
Name: cleaned_name, dtype: int64